# Get SDSS photometry for CGM$^2$ galaxy catalog

I am going to try `astroquery`

In [1]:
from astroquery.sdss import SDSS
from astropy import coordinates as coords
from astropy.table import Table
import astropy.units as u
import astropy
import numpy as np

/anaconda3/lib/python3.6/site-packages/astroquery/sdss/__init__.py:29: UserWarning: Experimental: SDSS has not yet been refactored to have its API match the rest of astroquery (but it's nearly there).
  warnings.warn("Experimental: SDSS has not yet been refactored to have its API "


#### Read the CGM$^2$ galaxy `ra` and `dec` from an `ascii` table I made

In [2]:
cgmsqrd_radec_table = Table.read('cgmsquared_gal_radec.txt', format='ascii')
cgmsqrd_radec_table

col1,col2
float64,float64
9.591402053833008,2.081559896469116
9.591021537780762,2.064729928970337
9.59142017364502,2.105020046234131
9.590646743774414,2.0493199825286865
9.591168403625488,2.1023199558258057
9.590909957885742,2.0942699909210205
9.590218544006348,2.0462799072265625
9.590399742126465,2.070280075073242
9.590591430664062,2.095560073852539


#### Convert the units to degrees. RA starts off in hourangle

In [3]:
ra = (cgmsqrd_radec_table['col1']*u.hourangle).to(u.degree)
dec = cgmsqrd_radec_table['col2']*u.degree

In [4]:
pos = coords.SkyCoord(ra, dec, frame='icrs')
pos

<SkyCoord (ICRS): (ra, dec) in deg
    [(143.87103081,  2.0815599 ), (143.86532307,  2.06472993),
     (143.8713026 ,  2.10502005), ..., (164.93828773, 14.72770023),
     (164.96542454, 14.72535992), (164.90010738, 14.72659016)]>

#### What fields do we want from SDSS? dered?

http://cas.sdss.org/dr7/en/help/browser/description.asp?n=PhotoObj&t=V

In [5]:
fields = ['ra', 'dec', 'u', 'err_u', 'g', 
          'err_g', 'r', 'err_r', 
          'i', 'err_i', 'z', 'err_z']

# add the dereddened mags just in case
dered_fields = ['dered_u', 'dered_g, ''dered_r', 'dered_i', 'dered_z']

fields += dered_fields

#### Run on one object to see how this works

In [6]:
sql_query = SDSS.query_crossid(pos, photoobj_fields=fields, get_query_payload=True)

In [7]:
# This will take too long and result in a broken pipe
# xid = SDSS.query_crossid(pos, photoobj_fields=['u', 'g', 'r', 'i', 'z'], timeout=100000)

#### Have to run a query on each object (!!!) since cant authorize with casjobs yet. Outputs to a list of tables

In [8]:
out_table = []
for coord in pos:
    xid = SDSS.query_crossid(coord, photoobj_fields=fields)
    out_table.append(xid)

/anaconda3/lib/python3.6/site-packages/astroquery/sdss/core.py:856: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  comments='#'))


In order to combine all the tables, need to get rid of the `None` elements in the list which come when there is no object in sdss. Convert to numpy as array, remove the Nones, convert back to list and stack all of em up. 

In [9]:
np_outtable = np.asarray(out_table)
good_elements = np_outtable[~(np_outtable == None)]
astropy.table.vstack([good_elements[2], good_elements[3]])

/anaconda3/lib/python3.6/site-packages/astropy/table/table.py:2653: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other


NameError: name 'astropy' is not defined

In [ ]:
sdss_ugriz_table = astropy.table.vstack(good_elements.tolist(), join_type='exact')

Save the file so we dont have to do this again

In [ ]:
sdss_ugriz_table.write("sdss_ugriz_table.fits", overwrite=True)